In [46]:
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import EfficientNet_V2_S_Weights
from tqdm.auto import tqdm


In [47]:
# Set the seed for reproducibility
#seed = 42
#random.seed(seed)
#torch.manual_seed(seed)

# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.mps.is_available():
    # ı am using mps for test this project
    device = torch.device('mps')
else:
    device = torch.device('cpu')

# data dircetory
data_dir = "dataset/"

# Globel variables
IMAGE_SIZE = 256
BATCH_SIZE = 32


In [48]:
from PIL import Image

class RemoveTopPixels:
    def __init__(self, pixels_to_remove):
        self.pixels_to_remove = pixels_to_remove

    def __call__(self, img):
        # Görüntünün boyutlarını al
        width, height = img.size
        # Üst 200 pikseli kırp
        cropped_img = img.crop((0, self.pixels_to_remove, width, height))
        return cropped_img

In [49]:
# transforms
transform = transforms.Compose([
    #RemoveTopPixels(200),
    transforms.Resize((100, 280)),
    #transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the data
dataset = ImageFolder(root=data_dir, transform=transform)
# Split the dataset into train, validation, and test sets

#dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.33, stratify=dataset.targets)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [50]:
model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT).to(device)
#model = models.mobilenet_v3_large(pretrained=True).to(device)
#model = TinyVGG(input_shape=3, hidden_units=64, output_shape=3).to(device)
#model = models.resnet50(pretrained=True).to(device)

"""model = SimpleViT(
    image_size = 256,
    patch_size = BATCH_SIZE,
    num_classes = 1000,
    dim = 512,
    depth = 6,
    heads = 16,
    mlp_dim = 1024
).to(device)"""

# Print model summary
#torchinfo.summary(model, input_size=(8, 3, IMAGE_SIZE, IMAGE_SIZE))

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#optimizer = optim.AdamW(model.parameters(), lr=1e-4)
#optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

num_epochs = 10  # Set to 10 for testing, adjust as needed

total_accuracy = 0
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss = 0.0

    # Training loop
    for inputs, targets in tqdm(train_loader, desc="Training Progress"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Evaluation
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            test_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    train_loss /= len(train_loader)  # Average training loss
    test_loss /= len(test_loader)  # Average test loss
    accuracy = 100 * correct / total  # Test accuracy
    total_accuracy += accuracy
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
#print(f"Average accuracy efficientnet_v2_s: {total_accuracy / num_epochs:.2f}%")


Training Progress: 100%|██████████| 5/5 [00:26<00:00,  5.30s/it]


Epoch 1/10, Training Loss: 7.1284, Test Loss: 6.2666, Accuracy: 1.43%


Training Progress: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it]


Epoch 2/10, Training Loss: 5.5796, Test Loss: 4.8496, Accuracy: 24.29%


Training Progress: 100%|██████████| 5/5 [00:05<00:00,  1.10s/it]


...

Epoch 8/10, Training Loss: 0.1088, Test Loss: 1.5198, Accuracy: 72.86%


Training Progress: 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


Epoch 9/10, Training Loss: 0.0543, Test Loss: 1.4611, Accuracy: 77.14%


Training Progress: 100%|██████████| 5/5 [00:07<00:00,  1.50s/it]


Epoch 10/10, Training Loss: 0.0600, Test Loss: 1.4672, Accuracy: 77.14%


In [51]:
print("Ml part with SVC")
feature_extractor = nn.Sequential(*list(model.children())[:-2])
model.eval()

X_train = []
y_train = []

X_Test = []
y_test = []

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        features = feature_extractor(inputs)
        features = features.view(features.size(0), -1)  # Flattening
        X_train.extend(features.cpu().numpy())
        y_train.extend(labels.cpu().numpy())
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        features = feature_extractor(inputs)
        features = features.view(features.size(0), -1)  # Flattening
        X_Test.extend(features.cpu().numpy())
        y_test.extend(labels.cpu().numpy())

X_train = np.array(X_train)
y_train = np.array(y_train)

X_Test = np.array(X_Test)
y_test = np.array(y_test)

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_Test_scaled = scaler.transform(X_Test)

del X_train, X_Test

Ml part with SVC


In [52]:
clf = SVC(kernel="rbf", C=1, gamma="scale")
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_Test_scaled)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)


Accuracy:  0.7285714285714285


In [53]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_Test_scaled)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.7857142857142857


In [54]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=seed)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_Test_scaled)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.7571428571428571


In [55]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_Test_scaled)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Accuracy:  0.7142857142857143
